In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/kaggle/input/kepler-exoplanet-search-results/cumulative.csv')
data

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,4.80600,0.63400,-0.63400,87.7,13.0,-13.0,1.11,0.32,-0.23,929.0,NaN,NaN,176.40,152.77,-77.60,8.4,1.0,q1_q17_dr25_tce,5638.0,169.0,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tce,6119.0,165.0,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,-0.002850,0.147,0.309,-0.147,0.86500,0.16200,-0.16200,103.6,14.7,-14.7,1.07,0.36,-0.11,2218.0,NaN,NaN,5713.41,5675.74,-1836.94,12

In [63]:
def preprocess_inputs(df):
    df = df.copy()
    
    ## Drop unused columns
    bad_cols = ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score']
    df = df.drop(bad_cols, axis = 1)
    
    ## Drop the false positive rows
    bad_rows = df.query("koi_disposition == 'FALSE POSITIVE'").index
    df = df.drop(bad_rows, axis = 0).reset_index(drop = True)
    
    ## Drop columns with all the missing values
    df = df.drop(['koi_teq_err1', 'koi_teq_err2'], axis = 1)
    
    ## Fill the categorical column Nan values
    df['koi_tce_delivname'] = df['koi_tce_delivname'].fillna(df['koi_tce_delivname'].mode()[0])
    
    ## Fill the numerical columns missing values
    for column in df.columns[df.isna().sum() > 0]:
        df[column] = df[column].fillna(df[column].mean())
        
    ## One-hot encode the koi_tce_delivname column
    delivname_dummies = pd.get_dummies(df['koi_tce_delivname'], prefix = 'dilivname')
    df = pd.concat([df, delivname_dummies], axis = 1)
    df = df.drop('koi_tce_delivname', axis = 1)
    
    ## Split the df into X and y
    y = df['koi_disposition']
    X = df.drop('koi_disposition', axis = 1)
    
    ## Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)
    
    ##Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [64]:
# X.columns

In [65]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)
X_train

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,dilivname_q1_q16_tce,dilivname_q1_q17_dr24_tce,dilivname_q1_q17_dr25_tce
1673,-0.071134,-0.117116,-0.0355,-0.017742,-0.037165,-0.239706,0.239706,-0.497521,-0.000669,0.000669,0.272742,-0.140167,-0.266272,-0.441041,-0.079904,0.079904,-0.155163,-0.279154,0.279154,-0.035572,-0.031034,0.027635,0.646748,-0.023607,-0.034229,0.020831,-0.215034,0.780456,0.833483,-0.138215,0.084673,-0.331827,0.332863,0.162643,-0.012212,-0.045685,0.033953,-1.053449,-1.878703,-0.325469,-0.242131,-0.185756,0.312051
1239,-0.071134,-0.117116,-0.0355,-0.017742,-0.004275,-0.054711,0.054711,0.202478,0.092227,-0.092227,-0.293536,-0.077331,0.298219,0.352551,-0.069836,0.069836,-0.083836,0.091315,-0.091315,-0.031127,-0.030643,0.026235,-0.844428,-0.027647,-0.041769,0.022613,-0.175524,-0.490423,-0.330867,-1.083094,1.023246,0.060167,0.171510,0.548779,-0.083330,-0.292382,0.077194,0.659043,0.429700,0.597714,-0.242131,-0.185756,0.312051
3589,-0.071134,-0.117116,-0.0355,-0.017742,0.174458,1.954787,-1.954787,-0.239901,0.177133,-0.177133,-0.156273,-0.110109,0.096010,0.318639,0.640932,-0.640932,-0.107305,0.157386,-0.157386,-0.013500,0.009134,0.012008,-0.907385,-0.027660,-0.041620,0.022611,-0.208317,-0.490423,-0.783183,-0.270058,0.408907,-2.906019,0.959291,-2.636846,0.476722,3.648441,-0.313997,-1.261863,-1.319194,-1.420679,4.129990,-0.185756,-3.204607
415,-0.071134,-0.117116,-0.0355,-0.017742,-0.037962,-0.253911,0.253911,0.110849,-0.411112,0.411112,-0.273988,-0.097572,0.269409,-0.502304,-0.487437,0.487437,0.128285,-0.162350,0.162350,-0.023041,-0.020479,0.024953,0.483060,-0.024695,-0.032533,0.021526,0.399749,-0.490423,0.273194,0.938508,-0.836835,0.455247,-0.511866,-0.947499,-0.096417,0.021787,0.102868,-1.024723,0.715114,1.253905,-0.242131,-0.185756,0.312051
2925,-0.071134,-0.117116,-0.0355,-0.017742,0.010035,0.559955,-0.559955,0.873931,0.966253,-0.966253,0.021674,-0.142886,-0.099716,2.651326,1.110078,-1.110078,-0.137152,-0.122235,0.122235,-0.032889,-0.030154,0.026002,-0.779673,-0.027628,-0.041722,0.022601,-0.205947,-0.490423,0.365117,-1.105068,1.040311,-0.470722,0.570146,0.210910,0.000876,-0.066771,0.012333,-0.261806,-1.248770,0.522987,-0.242131,-0.185756,0.312051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,-0.071134,-0.117116,-0.0355,-0.017742,-0.035650,-0.206766,0.206766,-0.396396,0.736509,-0.736509,-0.263603,-0.088207,0.254104,-0.019645,0.832215,-0.832215,-0.142028,0.029964,-0.029964,-0.035572,-0.029177,0.028335,0.123307,-0.026313,-0.037054,0.022072,-0.228863,-0.490423,0.960423,1.026404,-1.365849,0.297832,-0.454918,-1.092300,-0.080613,0.139864,0.090031,1.143456,1.093772,0.255217,-0.242131,-0.185756,0.312051
2763,-0.071134,-0.117116,-0.0355,-0.017742,-0.031798,-0.076652,0.076652,-0.276621,-0.121035,0.121035,-0.011924,-0.145605,-0.115921,-0.804897,-0.226889,0.226889,0.005323,1.535436,-1.535436,-0.025877,-0.021847,0.024136,-0.216659,-0.027136,-0.039964,0.022373,-0.210688,-0.490423,0.937078,1.509831,-1.741278,0.245360,-0.303056,-1.019900,-0.081848,0.095585,0.075842,0.628206,-1.259209,0.958113,-0.242131,-0.185756,0.312051
905,-0.071134,-0.117116,-0.0355,-0.017742,-0.026434,-0.229689,0.229689,-0.280512,-0.276861,0.276861,-0.294758,-0.086545,0.300019,0.372565,-0.321524,0.321524,-0.054763,-0.034928,0.034928,-0.026912,-0.028004,0.022853,-0.410926,-0.027394,-0.041346,0.022490,-0.107566,0.780456,0.264440,-0.292032,0.596622,-0.27626

In [66]:
X.koi_disposition.value_counts()

CONFIRMED    2293
CANDIDATE    2248
Name: koi_disposition, dtype: int64

In [47]:
X.query("koi_disposition == 'FALSE POSITIVE'").index

Int64Index([], dtype='int64')

In [48]:
(X.isna().mean() >= 0.25).sum()

0

In [49]:
X.columns[X.isna().sum() > 0]

Index([], dtype='object')

In [71]:
models = {
    "                   Logistic Regression": LogisticRegression(),
#     "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
#     "Support Vector Machine (Linear Kernel)": LinearSVC(),
#     "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
#     "                               XGBoost": XGBClassifier(eval_metric='logloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   Logistic Regression trained.
                         Decision Tree trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                              LightGBM trained.
                              CatBoost trained.


In [77]:
y_pred = model.predict(X_test)
y_pred

array(['CANDIDATE', 'CONFIRMED', 'CONFIRMED', ..., 'CONFIRMED',
       'CONFIRMED', 'CONFIRMED'], dtype=object)

In [84]:
def get_classifications(y_test, y_pred, positive_label='CONFIRMED'):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    
    for y_t, y_p in zip(y_test, y_pred):
        if y_t == positive_label:
            if y_p == positive_label:
                tp += 1
            else:
                fn += 1
        else:
            if y_p == positive_label:
                fp += 1
            else:
                tn += 1
    
    return tp, fn, fp, tn

def get_accuracy(tp, fn, fp, tn):
    acc = (tp + tn) / (tp + fn + fp + tn)
    return acc

def get_precision(tp, fn, fp, tn):
    precision = tp / (tp + fp)
    return precision

def get_recall(tp, fn, fp, tn):
    recall = tp / (tp + fn)
    return recall

def get_f1_score(tp, fn, fp, tn):
    precision = get_precision(tp, fn, fp, tn)
    recall = get_recall(tp, fn, fp, tn)
    f1_score = (2 * precision * recall) / (precision + recall)
    return f1_score

In [85]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name + " Accuracy: {:.3f}%".format(get_accuracy(*get_classifications(y_test, y_pred)) * 100))

                   Logistic Regression Accuracy: 79.751%
                         Decision Tree Accuracy: 75.569%
                        Neural Network Accuracy: 80.631%
                         Random Forest Accuracy: 81.878%
                     Gradient Boosting Accuracy: 82.539%
                              LightGBM Accuracy: 81.731%
                              CatBoost Accuracy: 82.392%


In [86]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name + " F1 Score: {:.5f}".format(get_f1_score(*get_classifications(y_test, y_pred))))

                   Logistic Regression F1 Score: 0.81673
                         Decision Tree F1 Score: 0.76231
                        Neural Network F1 Score: 0.81538
                         Random Forest F1 Score: 0.82811
                     Gradient Boosting F1 Score: 0.83609
                              LightGBM F1 Score: 0.82910
                              CatBoost F1 Score: 0.83516
